### Import thư viện

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

### Kiểm tra dữ liệu

In [13]:
data = pd.read_csv('../dataset/data.csv')
print(f"Kích thước dữ liệu: {data.shape}")
print(f"Số cột dữ liệu: {data.shape[1]}")
print(f"Số dòng dữ liệu: {data.shape[0]}")

print("\nMẫu dữ liệu:")
data.head()

Kích thước dữ liệu: (209417, 275)
Số cột dữ liệu: 275
Số dòng dữ liệu: 209417

Mẫu dữ liệu:


,diseases,anxiety and nervousness,depression,shortness of breath,depressive or psychotic symptoms,sharp chest pain,dizziness,insomnia,abnormal involuntary movements,chest tightness,...,skin pain,low back stiffness or tightness,skin on head or neck looks infected,nose deformity,sore in nose,hip weakness,back swelling,ankle stiffness or tightness,ankle weakness,neck weakness
0,vocal cord polyp,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,vocal cord polyp,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,vocal cord polyp,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,vocal cord polyp,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,vocal cord polyp,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Kiểm tra giá trị null

In [4]:
null_counts = data.isnull().sum()
print(f"Tổng số giá trị null: {null_counts.sum()}")

Tổng số giá trị null: 0


#### Số lượng bệnh và triệu chứng có trong mẫu

In [14]:
disease_counts = data['diseases'].value_counts()

symptom_cols = data.columns[1:] 
symptom_counts = data[symptom_cols].sum().sort_values(ascending=False)

print("Số lượng bệnh: ", disease_counts.shape[0])
print("Số triệu chứng: ", symptom_counts.shape[0])

Số lượng bệnh:  622
Số triệu chứng:  274


#### Danh sách bệnh

In [15]:
disease_list = disease_counts.index.tolist()
with open("../data_info/disease_list.txt", "w", encoding="utf-8") as f:
    for disease in disease_list:
        f.write(f"{disease}\n")
disease_list

['cystitis',
 'vulvodynia',
 'nose disorder',
 'spondylosis',
 'peripheral nerve disorder',
 'hypoglycemia',
 'esophagitis',
 'conjunctivitis due to allergy',
 'vaginal cyst',
 'diverticulitis',
 'gastrointestinal hemorrhage',
 'acute bronchitis',
 'fungal infection of the hair',
 'pneumonia',
 'sprain or strain',
 'infectious gastroenteritis',
 'gout',
 'strep throat',
 'marijuana abuse',
 'arthritis of the hip',
 'liver disease',
 'bursitis',
 'eczema',
 'acute pancreatitis',
 'spinal stenosis',
 'noninfectious gastroenteritis',
 'benign prostatic hyperplasia (bph)',
 'cholecystitis',
 'acute bronchiolitis',
 'obstructive sleep apnea (osa)',
 'concussion',
 'dental caries',
 'sebaceous cyst',
 'sickle cell crisis',
 'problem during pregnancy',
 'chronic constipation',
 'psoriasis',
 'pyogenic skin infection',
 'developmental disability',
 'seasonal allergies (hay fever)',
 'otitis media',
 'multiple sclerosis',
 'urinary tract infection',
 'sinus bradycardia',
 'contact dermatitis',


#### Danh sách triệu chứng

In [16]:
symptom_list = symptom_counts.index.tolist()
with open("../data_info/symptom_list.txt", "w", encoding="utf-8") as f:
    for symptom in symptom_list:
        f.write(f"{symptom}\n")
symptom_list

['vomiting',
 'sharp abdominal pain',
 'cough',
 'nausea',
 'sharp chest pain',
 'headache',
 'fever',
 'shortness of breath',
 'back pain',
 'nasal congestion',
 'dizziness',
 'skin swelling',
 'sore throat',
 'leg pain',
 'skin rash',
 'skin lesion',
 'burning abdominal pain',
 'lower abdominal pain',
 'weakness',
 'ear pain',
 'itching of skin',
 'diarrhea',
 'arm pain',
 'side pain',
 'loss of sensation',
 'pain in eye',
 'depressive or psychotic symptoms',
 'chest tightness',
 'low back pain',
 'peripheral edema',
 'diminished vision',
 'hip pain',
 'difficulty breathing',
 'coryza',
 'neck pain',
 'facial pain',
 'skin dryness, peeling, scaliness, or roughness',
 'abnormal involuntary movements',
 'acne or pimples',
 'suprapubic pain',
 'blood in stool',
 'painful urination',
 'lower body pain',
 'lacrimation',
 'heartburn',
 'knee pain',
 'pelvic pain',
 'decreased appetite',
 'itchiness of eye',
 'upper abdominal pain',
 'chills',
 'fainting',
 'shoulder pain',
 'paresthesia',


### Huấn luyện mô hình

#### Xử lý dữ liệu và chia tập train / test

In [17]:
X = data.iloc[:, 1:]  # Tất cả các cột trừ cột đầu tiên (diseases)
y = data.iloc[:, 0]   # Cột đầu tiên (diseases)

# Chuyển đổi nhãn bệnh thành số
le = LabelEncoder()
y_encoded = le.fit_transform(y)

print(f"Số lượng nhãn bệnh: {len(le.classes_)}")

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"Kích thước tập huấn luyện: {X_train.shape}")
print(f"Kích thước tập kiểm tra: {X_test.shape}")


Số lượng nhãn bệnh: 622
Kích thước tập huấn luyện: (167533, 274)
Kích thước tập kiểm tra: (41884, 274)


#### Huấn luyện bằng nhiều mô hình

In [18]:
models = {
    'Random Forest': RandomForestClassifier(n_estimators=50, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42)
}

# Huấn luyện và đánh giá các mô hình
results = {}

for name, model in models.items():
    print(f"\nĐang huấn luyện mô hình {name}...")
    model.fit(X_train, y_train)
    
    # Dự đoán trên tập kiểm tra
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Độ chính xác của {name}: {accuracy:.4f}")
    
    # Lưu kết quả
    results[name] = {
        'model': model,
        'accuracy': accuracy,
        'y_pred': y_pred
    }


Đang huấn luyện mô hình Random Forest...
Độ chính xác của Random Forest: 0.7969

Đang huấn luyện mô hình Decision Tree...
Độ chính xác của Decision Tree: 0.7822


#### So sánh và chọn mô hình tốt nhất

In [17]:
# Tìm mô hình tốt nhất
best_model_name = max(results, key=lambda x: results[x]['accuracy'])
best_model = results[best_model_name]['model']
best_accuracy = results[best_model_name]['accuracy']
best_y_pred = results[best_model_name]['y_pred']

print(f"Mô hình tốt nhất: {best_model_name} với độ chính xác {best_accuracy:.4f}")

# Ma trận nhầm lẫn
print("\nMa trận nhầm lẫn:")
cm = confusion_matrix(y_test, best_y_pred)
print(cm)
    
# Báo cáo chi tiết với precision, recall, F1-score
print("\nBáo cáo chi tiết (Precision, Recall, F1-score):")
report = classification_report(y_test, best_y_pred, zero_division=1)
print(report)


Mô hình tốt nhất: Random Forest với độ chính xác 0.8114

Ma trận nhầm lẫn:
[[28  0  0 ...  0  0  0]
 [ 0 71  0 ...  0  0  0]
 [ 0  0 41 ...  0  0  0]
 ...
 [ 0  0  0 ...  3  0  0]
 [ 0  0  0 ...  0 79  0]
 [ 0  0  0 ...  0  0  2]]

Báo cáo chi tiết (Precision, Recall, F1-score):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        28
           1       0.92      0.88      0.90        81
           2       0.58      0.71      0.64        58
           3       1.00      1.00      1.00         4
           4       0.84      0.93      0.88        68
           5       0.50      0.83      0.62         6
           6       0.31      0.57      0.40         7
           7       0.72      0.76      0.74        17
           8       0.44      0.55      0.49        99
           9       0.76      0.72      0.74       182
          10       0.81      0.83      0.82       241
          11       0.73      0.72      0.73       243
          12     

#### Lưu mô hình

In [18]:
# Tạo thư mục models nếu chưa tồn tại
os.makedirs('../models', exist_ok=True)

# Tối ưu lưu mô hình với nén
model_filename = f'../models/{best_model_name.replace(' ', '_').lower()}_model.joblib'
joblib.dump(best_model, model_filename, compress = 3) 

# Lưu label encoder cũng với nén
encoder_filename = '../models/label_encoder.joblib'
joblib.dump(le, encoder_filename, compress = 3)

print(f"Đã lưu mô hình tại: {model_filename}")
print(f"Đã lưu label encoder tại: {encoder_filename}")

Đã lưu mô hình tại: ../models/random_forest_model.joblib
Đã lưu label encoder tại: ../models/label_encoder.joblib
